In [1]:
# the classics
import pandas as pd
import numpy as np
import os 

# pyCO2sys
import PyCO2SYS as pyco2

# plotting
import matplotlib.pyplot as plt
# set plot params
plt.rcParams.update({'font.size': 16})
plt.rcParams.update({'lines.markersize': 5})
# make some colors
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']

This script uses PyCO2SYS to compare our measurements of pH, DIC, alkalinity, and pCO2. With two of these four paramters, the other two can be calculated using thermodynamic calculations. PyCO2SYS is nice because it propagates uncertainty. 

In [2]:
# here's the dataset
df = pd.read_csv('data/watersamples.csv',parse_dates=['datetime'])

<ipython-input-2-76cad0debe8c>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('data/watersamples.csv',parse_dates=['datetime'])


In [3]:
#### this plot is a pH-alk-DIC-pCO2 intercomparison. calculate everything from everything else
## calibrate -> validate
# ph-dic   ->  alk and pco2
# ph-alk   ->  dic and pco2
# ph-pco2   ->  dic and alk
# dic-alk   ->  ph and pco2
# dic-poc2   ->  ph and alk
# alk-pco2   ->  ph and dic


## begin with inputs to pyCO2sys
pco2_in = df['pCO2_ppmv']
dic_in = (df['DIC_mg_L']/12)*1000  # mg/L to umols/L
alk_in = df['Alkalinity_meq_L']*1000 # meq/L to ueq/L
ph_in = df['pH']
# for any temperatures missing, fill temp with mean
temp_in = df['temperature_deg_C'].fillna(np.mean(df['temperature_deg_C']))
# duplicate precisions: pyCO2sys takes all as relative (fractional)
dic_precision = 0.018
d13C_precision = 0.09227930700945312 # unused but included here for completeness
alk_precision = 0.1
ph_precision = 0.1
pco2_precision= 0.06

### number 1:
## ph-dic   ->  alk and pco2
df['pCO2_calc_dic_ph'] = pyco2.sys(par1=ph_in, par2=dic_in, par1_type=3, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['pCO2']
df['u_pCO2_calc_dic_ph'] = pyco2.sys(par1=ph_in, par2=dic_in, par1_type=3, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,                   
                    uncertainty_into=['pCO2'],
                    uncertainty_from={"par1": ph_precision,"par2__f": dic_precision})['u_pCO2']

df['alk_calc_dic_ph'] = pyco2.sys(par1=ph_in, par2=dic_in, par1_type=3, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['alkalinity']
df['u_alk_calc_dic_ph'] = pyco2.sys(par1=ph_in, par2=dic_in, par1_type=3, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['alkalinity'],
                    uncertainty_from={"par1": ph_precision,"par2__f": dic_precision})['u_alkalinity']

# save differences
alk_diff_dic_ph = 100*(df['alk_calc_dic_ph'].values-alk_in.values)/alk_in.values
alk_diff_dic_ph = alk_diff_dic_ph[~np.isnan(alk_diff_dic_ph)]

pco2_diff_dic_ph =100*(df['pCO2_calc_dic_ph'].values-pco2_in.values)/pco2_in.values
pco2_diff_dic_ph = pco2_diff_dic_ph[~np.isnan(pco2_diff_dic_ph)]


### number 2:
## ph-alk   ->  dic and pco2

df['dic_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['dic']
df['u_dic_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,                   
                    uncertainty_into=['dic'],
                    uncertainty_from={"par1": ph_precision,"par2__f": alk_precision})['u_dic']
df['pCO2_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['pCO2']
df['u_pCO2_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,                   
                    uncertainty_into=['pCO2'],
                    uncertainty_from={"par1": ph_precision,"par2__f": alk_precision})['u_pCO2']

# save differences
dic_diff_alk_ph =100*(df['dic_calc_alk_ph'].values-dic_in.values)/dic_in.values
dic_diff_alk_ph = dic_diff_alk_ph[~np.isnan(dic_diff_alk_ph)]

pco2_diff_alk_ph =100*(df['pCO2_calc_alk_ph'].values-pco2_in.values)/pco2_in.values
pco2_diff_alk_ph = pco2_diff_alk_ph[~np.isnan(pco2_diff_alk_ph)]


### number 3:
## ph-pco2   ->  dic and alk

df['dic_calc_pco2_ph'] = pyco2.sys(par1=pco2_in, par2=ph_in, par1_type=4, par2_type=3,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['dic']
df['u_dic_calc_pco2_ph'] = pyco2.sys(par1=pco2_in, par2=ph_in, par1_type=4, par2_type=3,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['dic'],
                    uncertainty_from={"par1__f": pco2_precision,"par2": ph_precision})['u_dic']

df['alk_calc_pco2_ph'] = pyco2.sys(par1=pco2_in, par2=ph_in, par1_type=4, par2_type=3,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['alkalinity']
df['u_alk_calc_pco2_ph'] = pyco2.sys(par1=pco2_in, par2=ph_in, par1_type=4, par2_type=3,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['alkalinity'],
                    uncertainty_from={"par1__f": pco2_precision,"par2": ph_precision})['u_alkalinity']

# save differences
dic_diff_pco2_ph =100*(df['dic_calc_pco2_ph'].values-dic_in.values)/dic_in.values
dic_diff_pco2_ph = dic_diff_pco2_ph[~np.isnan(dic_diff_pco2_ph)]

alk_diff_pco2_ph =100*(df['alk_calc_pco2_ph'].values-alk_in.values)/alk_in.values
alk_diff_pco2_ph = alk_diff_pco2_ph[~np.isnan(alk_diff_pco2_ph)]


### number 4:
## dic-alk   ->  ph and pco2
df['pH_calc_dic_alk'] = pyco2.sys(par1=alk_in, par2=dic_in, par1_type=1, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['pH']
df['u_pH_calc_dic_alk'] = pyco2.sys(par1=alk_in, par2=dic_in, par1_type=1, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['pH'],
                    uncertainty_from={"par1__f": alk_precision,"par2__f": dic_precision})['u_pH']

df['pCO2_calc_dic_alk'] = pyco2.sys(par1=alk_in, par2=dic_in, par1_type=1, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['pCO2']
df['u_pCO2_calc_dic_alk'] = pyco2.sys(par1=alk_in, par2=dic_in, par1_type=1, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['pCO2'],
                    uncertainty_from={"par1__f": alk_precision,"par2__f": dic_precision})['u_pCO2']

# save differences
ph_diff_dic_alk = 100*(df['pH_calc_dic_alk'].values-ph_in.values)/ph_in.values
ph_diff_dic_alk = ph_diff_dic_alk[~np.isnan(ph_diff_dic_alk)]

pco2_diff_dic_alk = 100*(df['pCO2_calc_dic_alk'].values-pco2_in.values)/pco2_in.values
pco2_diff_dic_alk = pco2_diff_dic_alk[~np.isnan(pco2_diff_dic_alk)]
 

### number 5:
## dic-poc2   ->  ph and alk

df['pH_calc_dic_pco2'] = pyco2.sys(par1=pco2_in, par2=dic_in, par1_type=4, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['pH']
df['u_pH_calc_dic_pco2'] = pyco2.sys(par1=pco2_in, par2=dic_in, par1_type=4, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['pH'],
                    uncertainty_from={"par1__f": alk_precision,"par2__f": dic_precision})['u_pH']

df['alk_calc_dic_pco2'] = pyco2.sys(par1=pco2_in, par2=dic_in, par1_type=4, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['alkalinity']
df['u_alk_calc_dic_pco2'] = pyco2.sys(par1=pco2_in, par2=dic_in, par1_type=4, par2_type=2,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['alkalinity'],
                    uncertainty_from={"par1__f": pco2_precision,"par2__f": dic_precision})['u_alkalinity']

# save differences
ph_diff_dic_pco2 =100*(df['pH_calc_dic_pco2'].values-ph_in.values)/ph_in.values
ph_diff_dic_pco2 = ph_diff_dic_pco2[~np.isnan(ph_diff_dic_pco2)]

alk_diff_dic_pco2 =100*(df['alk_calc_dic_pco2'].values-alk_in.values)/alk_in.values
alk_diff_dic_pco2 = alk_diff_dic_pco2[~np.isnan(alk_diff_dic_pco2)]


### number 6:
## alk-pco2   ->  ph and dic

df['ph_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['pH']
df['u_ph_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['pH'],
                    uncertainty_from={"par1__f": alk_precision,"par2__f": pco2_precision})['u_pH']

df['dic_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['dic']
df['u_dic_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8,
                    uncertainty_into=['dic'],
                    uncertainty_from={"par1__f": alk_precision,"par2__f": pco2_precision})['u_dic']

# save differences
ph_diff_alk_pco2 =100*(df['ph_calc_alk_pco2'].values-ph_in.values)/ph_in.values
ph_diff_alk_pco2 = ph_diff_alk_pco2[~np.isnan(ph_diff_alk_pco2)]

dic_diff_alk_pco2 =100*(df['dic_calc_alk_pco2'].values-dic_in.values)/dic_in.values
dic_diff_alk_pco2 = dic_diff_alk_pco2[~np.isnan(dic_diff_alk_pco2)]

TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [ ]:
# begin plot
# color scheme:
# ph-dic   ->  alk and pco2  blue
# ph-alk   ->  dic and pco2  orange
# ph-pco2   ->  dic and alk   green
# dic-alk   ->  ph and pco2   red
# dic-poc2   ->  ph and alk   purple
# alk-pco2   ->  ph and dic  brown

fig,ax=plt.subplots(4,2,figsize=(8,12),constrained_layout=True)
ax = ax.flatten()

## plot actual on x and predicted on y
# 1 calculated from DIC-pH
ax[2].errorbar(alk_in,df['alk_calc_dic_ph'],yerr=df['u_alk_calc_dic_ph'],ls='',marker='o',c=colors[0],zorder=2,markeredgecolor='k')
ax[6].errorbar(pco2_in,df['pCO2_calc_dic_ph'],yerr=df['u_pCO2_calc_dic_ph'],ls='',marker='o',c=colors[0],zorder=2,markeredgecolor='k')
# 2 calculated from alk-pH
ax[4].errorbar(dic_in,df['dic_calc_alk_ph'],yerr=df['u_dic_calc_alk_ph'],ls='',marker='o',c=colors[1],markeredgecolor='k')
ax[6].errorbar(pco2_in,df['pCO2_calc_alk_ph'],yerr=df['u_pCO2_calc_alk_ph'],ls='',marker='o',c=colors[1],zorder=2,markeredgecolor='k')
# 3 calculated from pco2-pH
ax[4].errorbar(dic_in,df['dic_calc_pco2_ph'],yerr=df['u_dic_calc_pco2_ph'],ls='',marker='o',c=colors[2],markeredgecolor='k')
ax[2].errorbar(alk_in,df['alk_calc_pco2_ph'],yerr=df['u_alk_calc_pco2_ph'],ls='',marker='o',c=colors[2],zorder=2,markeredgecolor='k')
# 4 calculated from dic-alk
ax[0].errorbar(ph_in,df['pH_calc_dic_alk'],yerr=df['u_pH_calc_dic_alk'],ls='',marker='o',c=colors[3],markeredgecolor='k')
ax[6].errorbar(pco2_in,df['pCO2_calc_dic_alk'],yerr=df['u_pCO2_calc_dic_alk'],ls='',marker='o',c=colors[3],zorder=2,markeredgecolor='k')
# 5 calculated from dic-pco2
ax[0].errorbar(ph_in,df['pH_calc_dic_pco2'],yerr=df['u_pH_calc_dic_pco2'],ls='',marker='o',c=colors[4],markeredgecolor='k')
ax[2].errorbar(alk_in,df['alk_calc_dic_pco2'],yerr=df['u_alk_calc_dic_pco2'],ls='',marker='o',c=colors[4],zorder=2,markeredgecolor='k')
# 6 calculated from alk-pco2
ax[0].errorbar(ph_in,df['ph_calc_alk_pco2'],yerr=df['u_ph_calc_alk_pco2'],ls='',marker='o',c=colors[5],markeredgecolor='k')
ax[4].errorbar(dic_in,df['dic_calc_alk_pco2'],yerr=df['u_dic_calc_alk_pco2'],ls='',marker='o',c=colors[5],markeredgecolor='k')

## make boxplots of differences
# pH
bplot1 = ax[1].boxplot([ph_diff_dic_alk,ph_diff_dic_pco2,ph_diff_alk_pco2],patch_artist=True,medianprops=dict(color='k'))
ax[1].set_xticks([1,2,3],['DIC-alk','DIC-pCO$_{2}$','alk-pCO$_{2}$'])
ax[1].set_ylabel('% error pH')
colors1 = ['tab:red','tab:purple','tab:brown']
for patch, color in zip(bplot1['boxes'], colors1):
    patch.set_facecolor(color)
# alk
bplot2 = ax[3].boxplot([alk_diff_dic_ph,alk_diff_pco2_ph,alk_diff_dic_pco2],patch_artist=True,medianprops=dict(color='k'))
ax[3].set_xticks([1,2,3],['pH-DIC','pH-pCO$_{2}$','DIC-pCO$_{2}$'])
ax[3].set_ylabel('% error alk')
colors2 = ['tab:blue','tab:green','tab:purple']
for patch, color in zip(bplot2['boxes'], colors2):
    patch.set_facecolor(color)
# dic
bplot3 = ax[5].boxplot([dic_diff_alk_ph,dic_diff_pco2_ph,dic_diff_alk_pco2],patch_artist=True,medianprops=dict(color='k'))
ax[5].set_xticks([1,2,3],['pH-alk','pH-pCO$_{2}$','alk-pCO$_{2}$'])
ax[5].set_ylabel('% error DIC')
colors3 = ['tab:orange','tab:green','tab:brown']
for patch, color in zip(bplot3['boxes'], colors3):
    patch.set_facecolor(color)
# pco2
bplot4 = ax[7].boxplot([pco2_diff_dic_ph,pco2_diff_alk_ph,pco2_diff_dic_alk],patch_artist=True,medianprops=dict(color='k'))
ax[7].set_xticks([1,2,3],['pH-DIC','pH-alk','DIC-alk'])
ax[7].set_ylabel('% error pCO$_{2}$')
colors4 = ['tab:blue','tab:orange','tab:red']
for patch, color in zip(bplot4['boxes'], colors4):
    patch.set_facecolor(color)
    
#### plot some 1:1 lines and adjust axes, label remaining axes
ax[0].plot([4,11],[4,11],c='k',ls='--',zorder=0)
ax[0].set_xlim([5,9])
ax[0].set_ylim([5,11])
ax[0].set_xlabel('pH measured')
ax[0].set_ylabel('pH modeled')

ax[2].plot([0,1500],[0,1500],c='k',ls='--',zorder=0)
ax[2].set_xlim([0,1000])
ax[2].set_ylim([0,2000])
ax[2].set_xlabel('alk measured ($\mu$Eq L$^{-1}$)')
ax[2].set_ylabel('alk modeled ($\mu$Eq L$^{-1}$)')

ax[4].plot([10,6000],[10,6000],c='k',ls='--',zorder=0)
ax[4].set_xlim([100,6000])
ax[4].set_ylim([100,6000])
ax[4].set_xscale('log')
ax[4].set_yscale('log')
ax[4].set_xlabel('DIC measured ($\mu$M)')
ax[4].set_ylabel('DIC modeled ($\mu$M)')
    
ax[6].plot([50,200000],[50,200000],c='k',ls='--',zorder=0)
ax[6].set_xlim([100,200000])
ax[6].set_ylim([100,200000])
ax[6].set_xscale('log')
ax[6].set_yscale('log')
ax[6].set_xlabel('pCO$_{2}$ measured (ppm)')
ax[6].set_ylabel('pCO$_{2}$ modeled (ppm)')

# horizontal lines at 0 for the boxplots
ax[1].axhline(0,c='k',ls='--',zorder=0)
ax[3].axhline(0,c='k',ls='--',zorder=0)
ax[5].axhline(0,c='k',ls='--',zorder=0)
ax[7].axhline(0,c='k',ls='--',zorder=0)


# save figure
fig.savefig('plots/FigS3_carbonatesystem.pdf')

It seems like DIC is parameter that disagrees the most with everything else. Especially the DIC-alk combo does not compare well with measured values. Given that DIC modeled is always lower than DIC measured, our DIC measurements are likely biased. So, we will depend on alk-pCO2 and alk-pH where pCO2 values were not measured. 

In [ ]:
#### now add some columns to our dataset, calculate the CO2, bicarb, and carb concentrations from alk-pco2 and alk-pH

df['KHenry'] = pyco2.sys(par1=ph_in, par2=dic_in, par1_type=3, par2_type=2,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['k_CO2']

# calculate pCO2 from pH and alk (this is what we go with to fill in missing pCO2)
df['pCO2_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['pCO2']

# calculate CO2 concentrations given pH-alk, alk-pCO2
df['CO2_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['CO2']/1000
df['CO2_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['CO2']/1000

# same for bicarb
df['bicarb_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['bicarbonate']/1000
df['bicarb_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['bicarbonate']/1000
# same for carb
df['carb_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['carbonate']/1000
df['carb_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature=temp_in,
                    opt_k_carbonic = 8)['carbonate']/1000

# same for dic
df['dic_calc_alk_ph'] = pyco2.sys(par1=ph_in, par2=alk_in, par1_type=3, par2_type=1,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['dic']
df['dic_calc_alk_pco2'] = pyco2.sys(par1=alk_in, par2=pco2_in, par1_type=1, par2_type=4,salinity=0,temperature = temp_in,
                    opt_k_carbonic=8)['dic']

# for samples with missing pCO2, calc from alk and pH (because we dont like DIC)
df['pco2'] = df['pCO2_ppmv'].fillna(df['pCO2_calc_alk_ph'])

# for all others, prioritize alk-pCO2, and fill the rest with alk and pH
df['CO2_calc'] = df['CO2_calc_alk_pco2'].fillna(df['CO2_calc_alk_ph'])
df['bicarb_calc'] = df['bicarb_calc_alk_pco2'].fillna(df['bicarb_calc_alk_ph'])
df['carb_calc'] = df['carb_calc_alk_pco2'].fillna(df['carb_calc_alk_ph'])
df['dic_calc'] = df['dic_calc_alk_pco2'].fillna(df['dic_calc_alk_ph'])
df['dic_calc_u'] = df['dic_calc_alk_pco2'].fillna(df['dic_calc_alk_ph'])

##### calculate isotope contents of individual DIC species, as outlined in Campeau et al., 2017
# isotope constants
e_carbog = np.exp((9878.83/(df['temperature_deg_C'])-22.183)/1000)-1
e_calg = 11.98 -0.12*df['temperature_deg_C']# from romanek 1992
df['e_bg'] = -0.1141*df['temperature_deg_C']+10.78   # bicarb - co2 gas
df['e_dg'] = 0.0049*df['temperature_deg_C']-1.31 # co2 aq - co2 gas
df['e_db'] = df['e_dg']-df['e_bg']
df['e_cg'] = -0.052*df['temperature_deg_C']+7.22
df['e_cb'] = df['e_cg']-df['e_bg']/(1+df['e_db']*10**-3)

# we only use the d13C-CO2, but it's nice to calculate the other species too
df['d13c_bicarb'] =( df['d13C_permil']*df['dic_calc']/1000-(df['e_db']*df['CO2_calc']+df['e_cb']*df['carb_calc']))/((1+df['e_db']*10**-3)*df['CO2_calc']+df['bicarb_calc']+(1+df['e_cb']*10**-3)*df['carb_calc'])
df['d13c_co2'] = df['d13c_bicarb']-(1+df['e_db']*10**-3)+df['e_db']
df['d13c_carb'] = df['d13c_bicarb']-(1+df['e_cb']*10**-3)+df['e_cb']

### before we export this dataset, merge the two samples of ER23-9
tempmerge = df.loc[df['sample_name'].isin(['ER23-9 6/6/23','ER23-9 6/13/23']),:].mean(numeric_only=True).to_frame().T
tempmerge['sample_name'] = 'ER23-9'
tempmerge['sample_type']='R'
tempmerge['datetime']=pd.Timestamp(2023,6,6)

df = df.loc[~df['sample_name'].isin(['ER23-9 6/6/23','ER23-9 6/13/23']),:]
dfout = pd.concat([df,tempmerge]).reset_index(drop=True)

In [ ]:
dfout.to_csv('data/watersamples_carbcalc.csv')